# Matrix Factorization
* Prediction is $\tilde R = UA^T$ 
* Loss fuction is $L = \lVert (R - \tilde R)^\Omega \rVert _2^2 + \lambda_u \lVert U \rVert _2^2 + \lambda_a \lVert A \rVert _2^2$
* $\Omega$ is the set of oberved pairs $(i, j)$
* $M^\Omega$ is the projection of $M$ onto $\Omega$ for any matrix $M$, that is $M_{ij}^\Omega$ is defined to be $M_{ij}$ when $(i, j) \in \Omega$ and $0$ otherwise
* $U$ is an $m x k$ matrix, $A$ is an $n x k$ matrix and $R$ is the $m x n$ ratings matrix

In [1]:
source = "MatrixFactorization";

In [2]:
using Random
using SparseArrays

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

# Alternating Least Squares Algorithm
* $u_{ik} = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - \tilde r_{ij} + u_{ik}a_{kj})}{\sum_{j \in \Omega_i} a_j^2 + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user
* Note that this equation is equivalent to solving $A^{\Omega_i} u_i = R^{\Omega_i}$ with $L_2$ regularization $\lambda_u$, where $\Omega_i = \{(i', j) \in \Omega | i' = i \}$

In [4]:
function make_prediction(users, items, U, A)
    r = zeros(eltype(U), length(users))
    @views Threads.@threads for i = 1:length(r)
        if (users[i] <= size(U)[1]) && (items[i] <= size(A)[1])
            r[i] = dot(U[users[i], :], A[items[i], :])
        end
    end
    r
end;

In [5]:
function calc_loss(df, U, A)
    loss = mse(df.rating, make_prediction(df.user, df.item, U, A))
    @debug "loss: $loss"
    loss
end;

In [6]:
function ridge_regression(X, y, λ)
    (Matrix(X'X) + λ * I(size(X)[2])) \ Vector(X'y)
end;

In [7]:
# julia matrices are column major by default so we take adjoints to make them row major
@memoize function sparse_csr(i, j, v, m, n)
    sparse(j, i, v, n, m)'
end;

@memoize function gaussian_init_csr(source, K, el_type)
    Random.seed!(20211204 * hash(source) * K)
    (zeros(el_type, K, maximum(source)) + randn(K, maximum(source)) * K^(-1 / 4))'
end;

In [8]:
function sparse_subset(A, rows)
    # returns a sparse matrix B such that: 
    # size(B) == size(A), B[rows, :] == A[rows, :], and B[~rows, :] == 0
    K = size(A)[2]
    nzval = vec(A[rows, :])
    rowval = repeat(rows, K)
    colptr = [1 + (x - 1) * length(rows) for x = 1:K+1]
    SparseMatrixCSC(size(A)..., colptr, rowval, nzval)
end;

In [9]:
function update_users!(users, items, ratings, U, A, λ_u)
    R = sparse_csr(users, items, ratings, size(U)[1], size(A)[1])
    Threads.@threads for i = 1:size(U)[1]
        X = sparse_subset(A, rowvals(R[i, :]))
        y = R[i, :]
        U[i, :] = ridge_regression(X, y, λ_u)
    end
end;

In [10]:
function train_model(training, λ_u, A, K, stop_criteria)
    @debug "training model with parameters [$λ_u]"
    users, items, ratings = training.user, training.item, training.rating
    U = copy(gaussian_init_csr(users, K, eltype(λ_u)))
    loss = calc_loss(training, U, A)

    while !stop!(stop_criteria, loss)
        update_users!(users, items, ratings, U, A, λ_u)
        loss = calc_loss(training, U, A)
    end
    U, loss
end;

In [11]:
mf_alphas = [
    x for
    x in read_params("CombineSignals")["alphas"] if occursin("MatrixFactorization", x)
]

3-element Vector{String}:
 "MatrixFactorization.10"
 "MatrixFactorization.20"
 "MatrixFactorization.40"

In [12]:
for alpha in mf_alphas
    params = read_params(alpha)
    training = get_residuals("recommendee", params["residual_alphas"])
    training.user .= 1 # relabel ids so that recommendee -> 1
    stop_criteria =
        early_stopper(max_iters = 100, patience = 1, min_rel_improvement = 0.0001)
    U, loss = train_model(training, params["λ"][1], params["A"], params["K"], stop_criteria)
    model(items) = make_prediction(fill(1, length(items)), items, U, params["A"])
    write_recommendee_alpha(model, outdir = alpha)
end

[ Debug: 20220107 02:01:32 training model with parameters [175.6941168065606]
[ Debug: 20220107 02:01:34 loss: 75.54485983887187
[ Debug: 20220107 02:01:35 loss: 2.335564287644249
[ Debug: 20220107 02:01:35 loss: 2.335564287644249
[ Debug: 20220107 02:02:06 training model with parameters [171.7920145176628]
[ Debug: 20220107 02:02:06 loss: 232.8279870656567
[ Debug: 20220107 02:02:06 loss: 2.113408770488716
[ Debug: 20220107 02:02:06 loss: 2.113408770488716
[ Debug: 20220107 02:02:06 training model with parameters [98.24458457647926]
[ Debug: 20220107 02:02:06 loss: 43.911005022133814
[ Debug: 20220107 02:02:06 loss: 1.9303105151983808
[ Debug: 20220107 02:02:06 loss: 1.9303105151983808
